# HW5 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  5
* **Date:**  6/17/16

## HW 5.0
- What is a data warehouse? What is a Star schema? When is it used?


## HW 5.1
- In the database world What is 3NF? Does machine learning use data in 3NF? If so why? 
- In what form does ML consume data?
- Why would one use log files that are denormalized?

## HW 5.2
Using MRJob, implement a hashside join (memory-backed map-side) for left, right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)

Justify which table you chose as the Left table in this hashside join.

**ANSWER:** There are 2 tables in this problem.  The first one is labeled 'PageID-URL' and it contains Page ID's and the URL's that the id's are associated with.  It contains about 590 records.  The second one is labeled 'PageID-VisitorID' and it contains Page ID's associated with Visitor ID's.  It contains about 130,000 records.  The smaller table (PageID-URL) was chosen to be on the left because when doing the left join there needs to be some way to mark which elements in the PageID-URL are not contained in the PageID-VisitorID table.  Doing this marking will take either additional time or additional space that is proportional to the size of the left table.  In order to minimize time and space, the smaller table was chosen to be on the left.

Please report the number of rows resulting from:

- (1) Left joining Table Left with Table Right
- (2) Right joining Table Left with Table Right
- (3) Inner joining Table Left with Table Right

**ANSWER:**

| Join Type | # of Rows |
| - | - |
| Left | xx |
| Right | xx |
| Inner | xx |


**Algorithm for hashside join** from Data-Intensive Text Processing with MapReduce by Jimmy Lin and Chris Dyer, section 3.5.3, page 67:  
1. Load the smaller dataset into memory in every mapper, populating an associative array to facilitate random access to tuples based on the join key. The mapper initialization API hook (see Section 3.1.1) can be used for this purpose.
2. Mappers are then applied to the other (larger) dataset, and for each input key-value pair, the mapper probes the in-memory dataset to see if there is a tuple with the same join key.
3. If there is, the join is performed.

#### (1) Left joining Table Left with Table Right

In [7]:
%%writefile MemJoinLeft.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
# This class performs left join on the 2 datasets in the input file.
# The left table is Page ID's and URL's.  The right table is Page_ID's and Visitor ID's.
# It will output all rows from the left table, with the matching rows (matching Page ID's)
# in the right table. The output is NULL or blank on the right side when there is no match.
# The output is of the form URL, Page ID, Visitor ID
class MRMemJoinLeft(MRJob):
    # Initialize a dictionary to store the smaller dataset that will be held in memory.
    vroots = {}
    # Initialize a dictionary to keep track of the vroots (page_id's) that did not exist
    # in the right table.
    vroots_not_used = {}
    
    def mapper_memjoin_init(self):
        # Read the data from the filename and store it in the self.vroots dictionary.  This
        # stores the base URL and the vroot labels for each vroot.  For each vroot
        # in the file, store an empty entry for the vroot in the self.vroots_not_used dictionary
        filename = 'anonymous-msweb_converted_small.data'
        with open(filename, 'r') as f:
            base_url = ""
            for line in f.readlines():
                record = line.strip().split(',')
                if record[0] == 'I':
                    base_url = record[2].strip('"')
                elif record[0] == 'A':
                    page_id = record[1]
                    vroot = record[4].strip('"')
                    self.vroots[page_id] = base_url + vroot
                    self.vroots_not_used[page_id] = ''

    def mapper_memjoin(self, _, line):
        # read the next line from the file and only if it is a visitor record, denoted by
        # 'V', and only if the page_id is in the vroots dictionary, output the URL, Page ID
        # and Visitor ID.  Delete the page_id from the self.vroots_not_used dictionary
        # to indicate that this page_id has been output already.
        record = line.strip().split(',')
        if record[0] == 'V':
            page_id = record[1]
            visitor_id = record[4]
            page_visitor_pair = ('Page ID: %s, Visitor ID: %s' % (page_id, visitor_id))
            if page_id in self.vroots:
                if page_id in self.vroots_not_used:
                    del self.vroots_not_used[page_id]
                yield 'URL: ' + self.vroots[page_id], page_visitor_pair

    def mapper_memjoin_final(self):
        # To complete the left join, for any vroots still left in the self.vroots_not_used
        # dictionary, print them out with a None for the right side.
        for page_id in self.vroots_not_used:
            yield 'URL: ' + self.vroots[page_id], None
    
    # Create the steps for this job.  No reducer is required.
    def steps(self):
        return[
            MRStep(mapper_init=self.mapper_memjoin_init, 
                   mapper=self.mapper_memjoin,
                   mapper_final=self.mapper_memjoin_final)
        ]

if __name__ == '__main__':
    MRMemJoinLeft.run()

Overwriting MemJoinLeft.py


In [8]:
!python MemJoinLeft.py anonymous-msweb_converted_small.data --file=anonymous-msweb_converted_small.data

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MemJoinLeft.hadoop.20160610.195536.152210
Running step 1 of 1...
Streaming final output from /tmp/MemJoinLeft.hadoop.20160610.195536.152210/output...
"URL: www.microsoft.com/regwiz"	"Page ID: 1000, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10001"
"URL: www.microsoft.com/athome"	"Page ID: 1002, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10003"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10003"
"URL: www.microsoft.com/search"	"Page ID: 1004, Visitor ID: 10003"
"URL: www.microsoft.com/norge"	"Page ID: 1005, Visitor ID: 10004"
"URL: www.microsoft.com/misc"	"Page ID: 1006, Visitor ID: 10005"
"URL: www.microsoft.com/kb"	"Page ID: 

In [9]:
import MemJoinLeft
reload(MemJoinLeft)

mr_job = MemJoinLeft.MRMemJoinLeft(args=['anonymous-msweb_converted_small.data','--file=anonymous-msweb_converted_small.data'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        if value == None:
            print key
        else:
            print key + ', ' + value
        count = count + 1
print "\n"
print "There are %s records" %count

URL: www.microsoft.com/regwiz, Page ID: 1000, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10001
URL: www.microsoft.com/athome, Page ID: 1002, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10003
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10003
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10003
URL: www.microsoft.com/norge, Page ID: 1005, Visitor ID: 10004
URL: www.microsoft.com/misc, Page ID: 1006, Visitor ID: 10005
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10006
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10006
URL: www.microsoft.com/ie_intl, Page ID: 1007, Visitor ID: 10007
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10008
URL: www.microsoft.com/msdownload, Page ID: 1008, 

#### (2) Right joining Table Left with Table Right

In [10]:
%%writefile MemJoinRight.py
from mrjob.job import MRJob
from mrjob.step import MRStep

# This class performs right join on the 2 datasets in the input file.
# The left table is Page ID's and URL's.  The right table is Page_ID's and Visitor ID's.
# It will output all rows from the right table, with the matching rows (matching Page ID's)
# in the left table. The output is NULL or blank on the left side when there is no match.
# The output is of the form URL, Page ID, Visitor ID
class MRMemJoinRight(MRJob):
    # Create a dictionary to store the smaller dataset that will be held in memory.
    vroots = {}
    
    def mapper_memjoin_init(self):
        # Read the data from the filename and store it in the self.vroots dictionary.  This
        # stores the base URL and the vroot labels for each vroot. 
        filename = 'anonymous-msweb_converted_small.data'
        with open(filename, 'r') as f:
            base_url = ""
            for line in f.readlines():
                record = line.strip().split(',')
                if record[0] == 'I':
                    base_url = record[2].strip('"')
                elif record[0] == 'A':
                    page_id = record[1]
                    vroot = record[4].strip('"')
                    self.vroots[page_id] = base_url + vroot

    def mapper_memjoin(self, _, line):
        # read the next line from the file examine each line that starts with a 'V'.
        # Yield each of these lines.  If there is no url for one of the lines in the 
        # file, then output it with a url of 'None'.
        record = line.strip().split(',')
        if record[0] == 'V':
            page_id = record[1]
            visitor_id = record[4]
            page_visitor_pair = ('Page ID: %s, Visitor ID: %s' % (page_id, visitor_id))
            url = 'None'
            if page_id in self.vroots:
                url = self.vroots[page_id]
            yield 'URL: ' + url, page_visitor_pair
    
    # Create the steps for this job.  No reducer is required.
    def steps(self):
        return[
            MRStep(mapper_init=self.mapper_memjoin_init, 
                   mapper=self.mapper_memjoin)
        ]

if __name__ == '__main__':
    MRMemJoinRight.run()

Overwriting MemJoinRight.py


In [11]:
!python MemJoinRight.py anonymous-msweb_converted_small.data --file=anonymous-msweb_converted_small.data

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MemJoinRight.hadoop.20160610.195612.500984
Running step 1 of 1...
Streaming final output from /tmp/MemJoinRight.hadoop.20160610.195612.500984/output...
"URL: www.microsoft.com/regwiz"	"Page ID: 1000, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10001"
"URL: www.microsoft.com/athome"	"Page ID: 1002, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10003"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10003"
"URL: www.microsoft.com/search"	"Page ID: 1004, Visitor ID: 10003"
"URL: www.microsoft.com/norge"	"Page ID: 1005, Visitor ID: 10004"
"URL: www.microsoft.com/misc"	"Page ID: 1006, Visitor ID: 10005"
"URL: www.microsoft.com/kb"	"Page ID

In [12]:
import MemJoinRight
reload(MemJoinRight)

mr_job = MemJoinRight.MRMemJoinRight(args=['anonymous-msweb_converted_small.data','--file=anonymous-msweb_converted_small.data'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key + ', ' + value
        count = count + 1
print "\n"
print "There are %s records" %count

URL: www.microsoft.com/regwiz, Page ID: 1000, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10001
URL: www.microsoft.com/athome, Page ID: 1002, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10003
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10003
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10003
URL: www.microsoft.com/norge, Page ID: 1005, Visitor ID: 10004
URL: www.microsoft.com/misc, Page ID: 1006, Visitor ID: 10005
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10006
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10006
URL: www.microsoft.com/ie_intl, Page ID: 1007, Visitor ID: 10007
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10008
URL: www.microsoft.com/msdownload, Page ID: 1008, 

#### (3) Inner joining Table Left with Table Right

In [13]:
%%writefile MemJoinInner.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
# This class performs an inner join on the 2 datasets in the input file.
# The left table is Page ID's and URL's.  The right table is Page_ID's and Visitor ID's.
# It will only output rows where the Page ID exists in both tables.  The output is 
# of the form URL, Page ID, Visitor ID
class MRMemJoinInner(MRJob):
    # Create a dictionary to store the smaller dataset that will be held in memory.
    vroots = {}
    
    def mapper_memjoin_init(self):
        # Read the data from the filename and store it in the self.vroots dictionary.  This
        # stores the base URL and the vroot labels for each vroot. 
        filename = 'anonymous-msweb_converted_small.data'
        with open(filename, 'r') as f:
            base_url = ""
            for line in f.readlines():
                record = line.strip().split(',')
                if record[0] == 'I':
                    base_url = record[2].strip('"')
                elif record[0] == 'A':
                    page_id = record[1]
                    vroot = record[4].strip('"')
                    self.vroots[page_id] = base_url + vroot

    def mapper_memjoin(self, _, line):
        # read the next line from the file and only if it is a visitor record, denoted by
        # 'V', and only if the page_id is in the vroots dictionary, output the URL, Page ID
        # and Visitor ID.
        record = line.strip().split(',')
        if record[0] == 'V':
            page_id = record[1]
            visitor_id = record[4]
            page_visitor_pair = ('Page ID: %s, Visitor ID: %s' % (page_id, visitor_id))
            if page_id in self.vroots:
                yield 'URL: ' + self.vroots[page_id], page_visitor_pair
    
    # Create the steps for this job.  No reducer is required.
    def steps(self):
        return[
            MRStep(mapper_init=self.mapper_memjoin_init, 
                   mapper=self.mapper_memjoin)
        ]

if __name__ == '__main__':
    MRMemJoinInner.run()

Overwriting MemJoinInner.py


In [15]:
!python MemJoinInner.py anonymous-msweb_converted_small.data --file=anonymous-msweb_converted_small.data

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MemJoinInner.hadoop.20160610.195911.904595
Running step 1 of 1...
Streaming final output from /tmp/MemJoinInner.hadoop.20160610.195911.904595/output...
"URL: www.microsoft.com/regwiz"	"Page ID: 1000, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10001"
"URL: www.microsoft.com/athome"	"Page ID: 1002, Visitor ID: 10001"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10002"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10002"
"URL: www.microsoft.com/support"	"Page ID: 1001, Visitor ID: 10003"
"URL: www.microsoft.com/kb"	"Page ID: 1003, Visitor ID: 10003"
"URL: www.microsoft.com/search"	"Page ID: 1004, Visitor ID: 10003"
"URL: www.microsoft.com/norge"	"Page ID: 1005, Visitor ID: 10004"
"URL: www.microsoft.com/misc"	"Page ID: 1006, Visitor ID: 10005"
"URL: www.microsoft.com/kb"	"Page ID

In [30]:
import MemJoinInner
reload(MemJoinInner)

mr_job = MemJoinInner.MRMemJoinInner(args=['anonymous-msweb_converted_small.data','--file=anonymous-msweb_converted_small.data'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key + ', ' + value
        count = count + 1
print "\n"
print "There are %s records" %count

URL: www.microsoft.com/regwiz, Page ID: 1000, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10001
URL: www.microsoft.com/athome, Page ID: 1002, Visitor ID: 10001
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10002
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10002
URL: www.microsoft.com/support, Page ID: 1001, Visitor ID: 10003
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10003
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10003
URL: www.microsoft.com/norge, Page ID: 1005, Visitor ID: 10004
URL: www.microsoft.com/misc, Page ID: 1006, Visitor ID: 10005
URL: www.microsoft.com/kb, Page ID: 1003, Visitor ID: 10006
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10006
URL: www.microsoft.com/ie_intl, Page ID: 1007, Visitor ID: 10007
URL: www.microsoft.com/search, Page ID: 1004, Visitor ID: 10008
URL: www.microsoft.com/msdownload, Page ID: 1008, 

## HW 5.3  EDA of Google n-grams dataset
A large subset of the Google n-grams dataset

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket/folder on Dropbox on s3:

https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACUifrl6wrMrlK6a3X3lZ9Ea?dl=0 

s3://filtered-5grams/

In particular, this bucket contains (~200) files (10Meg each) in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

For HW 5.3-5.5, for the Google n-grams dataset unit test and regression test your code using the 
first 10 lines of the following file:

googlebooks-eng-all-5gram-20090715-0-filtered.txt

Once you are happy with your test results proceed to generating  your results on the Google n-grams dataset. 

Do some EDA on this dataset using mrjob, e.g., 

- Longest 5-gram (number of characters)
- Top 10 most frequent words (please use the count information), i.e., unigrams
- 20 Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency 
- Distribution of 5-gram sizes (character length).  E.g., count (using the count field) up how many times a 5-gram of 50 characters shows up. Plot the data graphically using a histogram.

## HW 5.3.1 OPTIONAL Question:
Plot the log-log plot of the frequency distributuion of unigrams. Does it follow power law distribution?

For more background see:
- https://en.wikipedia.org/wiki/Log%E2%80%93log_plot
- https://en.wikipedia.org/wiki/Power_law



## HW 5.4  Synonym detection over 2Gig of Data

For the remainder of this assignment you will work with two datasets:

### 1: unit/systems test data set: SYSTEMS TEST DATASET
Three terms, A,B,C and their corresponding strip-docs of co-occurring terms

- DocA {X:20, Y:30, Z:5}
- DocB {X:100, Y:20}
- DocC {M:5, N:20, Z:5}

### 2: A large subset of the Google n-grams dataset as was described above

For each HW 5.4 -5.5.1 Please unit test and system test your code with respect 
to SYSTEMS TEST DATASET and show the results. 
Please compute the expected answer by hand and show your hand calculations for the 
SYSTEMS TEST DATASET. Then show the results you get with you system.

In this part of the assignment we will focus on developing methods
for detecting synonyms, using the Google 5-grams dataset. To accomplish
this you must script two main tasks using MRJob:

(1) Build stripes for the most frequent 10,000 words using cooccurence informationa based on
the words ranked from 9001,-10,000 as a basis/vocabulary (drop stopword-like terms),
and output to a file in your bucket on s3 (bigram analysis, though the words are non-contiguous).


(2) Using two (symmetric) comparison methods of your choice 
(e.g., correlations, distances, similarities), pairwise compare 
all stripes (vectors), and output to a file in your bucket on s3.

==Design notes for (1)==
For this task you will be able to modify the pattern we used in HW 3.2
(feel free to use the solution as reference). To total the word counts 
across the 5-grams, output the support from the mappers using the total 
order inversion pattern:

<*word,count>

to ensure that the support arrives before the cooccurrences.

In addition to ensuring the determination of the total word counts,
the mapper must also output co-occurrence counts for the pairs of
words inside of each 5-gram. Treat these words as a basket,
as we have in HW 3, but count all stripes or pairs in both orders,
i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

==Design notes for (2)==
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

- Jaccard
- Cosine similarity
- Spearman correlation
- Euclidean distance
- Taxicab (Manhattan) distance
- Shortest path graph distance (a graph, because our data is symmetric!)
- Pearson correlation
- Kendall correlation

However, be cautioned that some comparison methods are more difficult to
parallelize than others, and do not perform more associations than is necessary, 
since your choice of association will be symmetric.

Please use the inverted index (discussed in live session #5) based pattern to compute the pairwise (term-by-term) similarity matrix. 

Please report the size of the cluster used and the amount of time it takes to run for the index construction task and for the synonym calculation task. How many pairs need to be processed (HINT: use the posting list length to calculate directly)? Report your  Cluster configuration!



## HW 5.5 Evaluation of synonyms that your discovered
In this part of the assignment you will evaluate the success of you synonym detector (developed in response to HW5.4).
Take the top 1,000 closest/most similar/correlative pairs of words as determined by your measure in HW5.4, and use the synonyms function in the accompanying python code:

nltk_synonyms.py

Note: This will require installing the python nltk package:

http://www.nltk.org/install.html

and downloading its data with nltk.download().

For each (word1,word2) pair, check to see if word1 is in the list, 
synonyms(word2), and vice-versa. If one of the two is a synonym of the other, 
then consider this pair a 'hit', and then report the precision, recall, and F1 measure  of 
your detector across your 1,000 best guesses. Report the macro averages of these measures.

In [4]:
d = {}
d[1] = ''
print(d)

{1: ''}
